# Setup
Before we proceed, we need to... 
- import any necessary libraries
- create a function to help display API responses
- and fill out our config oject

In [ ]:
import requests
import json
import random

def view(data):
    print(json.dumps(data, indent=4))

### Fill in the `config` object below with your information
- `version` is the LightningDocs API version you wish to use for the translator.
- `tenancy` should be the name of the Knackly workspace.
- `keyID` is the Knackly API key provided to you.
- `secret` is the very long Knackly secret key provided to you.

Afterwards, make sure to run the cell. This ensures that all the variables are properly stored in memory.

In [ ]:
from util import LightningDocs, Knackly, Config

config = Config(
    LightningDocs(version="0.0.1"),
    Knackly(
        tenancy="testingapigeraci",
        keyID="YOUR_KEYID_HERE",
        secret="YOUR_SECRET_HERE",
        catalog_name="Test_Client",
        app_name="LoanDocuments",
        refresh_token="YOUR_REFRESH_TOKEN_HERE"
    ),
)

base_url = f"https://api.knackly.io/{config.knackly.tenancy}/api/v1"

# Interacting with the LightningDocs API
- _Schema documentation can be found [here](https://lightningdocs.com/)_ (<- update this to be the real link as soon as the correct one becomes available)
- _Postman documentation for the API can be found [here](https://lightningdocs.com/)_ (<- update this to be the real link as soon as the correct one becomes available)

### Sending valid JSON and getting back valid Knackly JSON

The following cell sends the contents of `valid.json` which follows the provided schema with information for a completely filled out loan.

In [ ]:
url = "https://api.lightningdocs.com/api/JsonConversion/Convert"

payload = {
    'version': config.lightningdocs.version
}

with open("json/lightningdocs/valid.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response.json())

The following cell uses data from `incomplete.json`. This is valid JSON that follows the schema, but is largely incomplete.

In [ ]:
with open("json/lightningdocs/incomplete.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response.json())

### Sending irrelevant / invalid JSON...

#### Sending data that is irrelevant
Irrelevant data will simply be ignored by the API. Only relevant information will be converted.

In [ ]:
with open("json/lightningdocs/irrelevant.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response.json())

#### Sending data that does not follow the schema
Validating your data against the schema is *your* responsibility. The API does not check the sent data against the schema, so any values that are not in the format Knackly expects **will not** be corrected.

For example, in the following cell file `invalid.json` is used. According to the schema, the `closingDate` key should follow the `date` format *("YYYY-MM-DD")*. However, we are sending a value of "January 14th, 2001".

In [ ]:
with open("json/lightningdocs/invalid.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response.json())

# Interacting with the Knackly API
Nearly all requests to the Knackly API must contain an access token provided in the request header. In order to obtain this access token, we will need the `keyID` and `secret` provided to us by LightningDocs.

## Access Token

In [ ]:
url = f"https://api.knackly.io/{config.knackly.tenancy}/api/v1/auth/login"

payload = {"KeyID": config.knackly.keyID, "Secret": config.knackly.secret}

response = requests.post(url, json=payload)
access_token = response.json()["token"]

view(response.json())

The access token needs to be included in the header of each subsequent response, so we will create the `headers` object which will be included in nearly all future requests.

In [ ]:
headers = {"Authorization": f"Bearer {access_token}"}

## External Users
External users are how you can allow users access to the Knackly system without the need to register and log in. There area two components of an external user. The `refresh token` and the `user token`.

We (LightningDocs) will provide you with a `refresh token` for an external user account. This is analagous to a recovery key for the external user: the `refresh token` itself isn't the "password" to the external token, but possession of it allows generation of the `user token`, which _is_ used to uniquely identify an external user.

### Refreshing an external user's `Token`
The `user token` for each external user is time sensitive, and will expire after a certain period of time. If the user token is found to be invalid, you can use the provided refresh token to generate a new, valid token for the associated external user.

In [ ]:
url = f"{base_url}/externals/refresh"

payload = {"Refresh_Token": config.knackly.refresh_token}

response = requests.post(url, headers=headers, json=payload)
user_token = response.json()["Token"]

view(response.json())

### Validating an external user
It is a good idea to validate that the external user's token is valid before attempting to make other API calls involving that user.

- If valid, the body of the GET response will be empty.
- If invalid, the body of the GET response will contain an error message.

In [ ]:
# Valid user
url = f"{base_url}/externals/valid/{user_token}"

response = requests.get(url, headers=headers)
response.text

In [ ]:
# Invalid user example
invalid_user_token = "obviouslyIncorrectToken123"
url = f"{base_url}/externals/valid/{invalid_user_token}"

response=requests.get(url, headers=headers)
response.text

### Creating a new record on a specific app to be accessed by an external user
Note that in this example, we are sending information `payload` along with the API request, in order to have parts of the interview already filled out. This step is optional, and you could opt not to send any information, resulting in a completely blank interview if you would like your users to start from scratch.

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/apps/{config.knackly.app_name}"

params = {
    "external": user_token
}

payload = {
    "lender": {
        "id$": "1",
        "name": "Larry the Lender 2",
        "address": {
            "id$": "2",
            "city": "Lafayette",
            "street": "1234 Laniakae Lane",
            "zip": "98765",
            "state": "California",
        },
    }
}

response = requests.post(url, params=params, json=payload, headers=headers)

view(response.json())

Now that the record has been created, we will use the `url` returned to us as part of the `apps` array in the API response to build a new url which can be given to the external user, allowing them to open up the interview and edit this record in their browser. 

Note the `return_complete` and `return_incomplete` parameters in the cell below. These are optional, but will redirect the user's browser to the associated url when they hit either "Complete" or "Finish Later" from the interview screen, respectively. In this example we are redirecting to a youtube video, but in a real application this url could be itself an API call that would alert your system that the user has finished their work, from which you can then do other stuff.

In [ ]:
apps_for_record = response.json()["apps"] # This is an array of app objects. Since our catalog only has one app, our array has a length of 1
app_url = apps_for_record[0]["url"]

from urllib.parse import urlencode
params = {
    'external': user_token,
    "return_complete": "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "return_incomplete": "https://example.com/"
}
external_url = f"{app_url}?{urlencode(params)}"

print(external_url)

### Checking status and getting record details
Regardless of whether the user ends up hitting "Finish Later" or "Complete" on the interview, you likely want to check the status of the app and obtain details about the record.

#### GET - Get status of an app on a record
Part of the response in the API call to create a record is the record's `id`. This is used to reference that specific record, and will be useful in future calls.

In [ ]:
record_id = response.json()["id"]

url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}/apps/{config.knackly.app_name}"

response = requests.get(url, headers=headers)

view(response.json())

#### GET - Get record details
Checking the status of an app will return information including URLs to the downloadable documents, but you can retrieve more detailed information about the record if necessary.

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"

response = requests.get(url, headers=headers)

view(response.json())

As you can see above, part of the response from this API call is the `files` array. Each element in this array is an object containing:
- `name` the actual filename of the downloadable document.
- `url` the ??????????????????.
- `publicURL` the link to downloadble document.

### Webhooks
Knackly supports webhooks, meaning you can receive POST requests _from_ Knackly to your own program, and then do whatever you want with the provided data. As of now, webhooks will only fire on the `catalog.app.completed` event, which is triggered whenever any app in a given catalog is completed.

The data that is included with each event is as follows:

| property | description |
| --- | --- |
| eventId	| a unique ID identifying this event (not currently used for much) |
| eventName	| the name of the event -- currently always "catalog.app.completed" |
| catalog	| the name of the Knackly catalog in which the event occurred |
| record	| the ID of the record in the above catalog, on which an app was completed. Use this to retrieve data or documents. |
| app	| the name of the app that was completed |
| docsStarted	| the date/time stamp when the user completed the app (documents started generating) |
| docsFinished	| the date/time stamp when documents finished generating for the app |
| docCount	| the number of documents generated by the app |
| userType	| either "regular", "external", or "api" |
| userName	| the user's name (for regular or external users) or the API key name (if app was run directly from the API) |
| userEmail	| the user's email address (for regular and external users only) |

#### Get webhooks
Optionally, an `id` can be passed to the url to get information a specific webhook

In [ ]:
url = f"{base_url}/webhooks"

response = requests.get(url, headers=headers)

view(response.json())

#### Register a webhook

In [ ]:
url = f"{base_url}/webhooks"

data = {
    "url": "YOUR_URL_HERE",
    "events": [
        "catalog.app.completed"
    ],
    "catalogs": [
        config.knackly.catalog_name
    ]
}

response = requests.post(url, headers=headers, json=data)

response.text

#### Unregister a webhook

In [ ]:
url = f"{base_url}/webhooks/YOUR_WEBHOOK_ID_HERE"

response = requests.delete(url, headers=headers)

response.text

## Records
Knackly provides an easy way to retrieve all records from a given catalog.

### Get all records in a catalog

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items"

payload = {
    "status": "Ok", # Optional parameter to filter results to only contain records with a particular status
}

response = requests.get(url, headers=headers, params=payload)

view(response.json())

As you can see, the above API call only returns **metadata** about the records in a catalog. In order to get more specific details for a given record, you must do that through an individual API call.

### Get details for a specific record

In [ ]:
record_id = random.choice(response.json())["id"] # In practice, this would be a specific id. A random choice of the valid id's is made here for demonstration purposes

url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"

new_response = requests.get(url, headers=headers)

view(new_response.json())

The structure of the response is split into three sections:
- `metadata` information about the record itself, such as a unique id, status, and last modified date
- `data` the actual data / answers given in the record
- `apps` any apps that pertain to this record, including URLs to any downloadable documents produced by the apps

### Possible Use Cases
The following cells show some examples of what is possible with record details. Every application has different needs and may be written in different programming languages. These are merely to give you ideas; you will need to write your own code specific to your own needs.

#### Retrieving all of the Downloadable Documents Produced After a Specified Date

In [ ]:
from datetime import datetime

# Step 1 - get a list of all the records in the catalog that have a status of "Ok", and modified after October 15th, 2023.
threshold_date = datetime(year=2023, month=10, day=15)

url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items"
payload = {
    "status": "Ok"
}
response = requests.get(url, headers=headers, params=payload)

all_record_ids = [record["id"] for record in response.json() if datetime.strptime(record["lastModified"], '%Y-%m-%dT%H:%M:%S.%fZ') > threshold_date]

# Step 2 - retrieve the record details for each of the record ids
all_downloadable_urls = []
for record_id in all_record_ids:
    url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"
    response = requests.get(url, headers=headers)
    
    # Step 3 - for each app, look at at the "files" array and store any "publicUrl"s found for this record
    apps = response.json()["apps"]
    for app in apps:
        files = app["files"]
        urls = [file["publicUrl"] for file in files if "publicUrl" in file]
        all_downloadable_urls.extend(urls)
        
# Step 4 - view all the urls
for url in all_downloadable_urls:
    print(url)
        

#### Retrieving a count of all the records last modified by a particular user which still need updating

In [ ]:
# Step 1 - get a list of all records in the catalog that need updating
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items"
payload = {
    "status": "Needs Updating"
}
response = requests.get(url, headers=headers, params=payload)
all_records = response.json()

# Step 2 - create a new list which only include records last modified by a particular user
user_to_search_for = "Joey Smith"
filtered_records = [record for record in all_records if record["lastModifiedBy"] == user_to_search_for]

# Step 3 - print out the number of filtered records
len(filtered_records)

# Summary
The combination of the LightningDocs and Knackly API is powerful, and opens up many avenues for creating unique programs. In one fell swoop, you can send JSON data to the LightningDocs API and receive Knackly-ready data, ship it off to Knackly, and download the resulting documents to a local machine. Someone can write a better summary here eventually, but for now this will do I think.

### Contact Information
If you are having trouble with any of the steps above, feel free to reach out with questions

- Trevor Kneisly - t.kneisly@lightningdocs.com
- Seth Collins - s.collins@lightningdocs.com

### Useful Links
- [LightningDocs API schema](https://lightningdocs.com/)
- [LightningDocs API documentation](https://lightningdocs.com/)
- [Knackly API documentation](https://documenter.getpostman.com/view/6868588/SzS7QReU#intro)

# --- Temporary stuff below here (WILL BE DELETED) ---

### Creating an external user (HIDE THIS AT THE END)

In [ ]:
url = f"{base_url}/externals"

payload = {"Email": "loadsadl@sz.com", "Name": "Joey Smith"}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    view(response.json())
else:
    print(response.text)

* If you get the message `'Error: Email is already associated with a user'`, then change the value of the `Email` or `Name` key in the `payload` object, and re-run the cell.

In [ ]:
user_token = response.json()["Token"]
user_refresh = response.json()["Refresh_Token"]